In [132]:
import pandas as pd
import numpy as np
import time
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor    
from lifelines import WeibullAFTFitter


In [133]:
data = pd.read_pickle('ALLDATA.pkl')
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Clear', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Day', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_40 - 49', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'Education_kid', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'shopwalk_kid', 'Vrexp_Yes', 'Heart_Currently',
                   'Heart_Over the years', 'vision_Currently', 'vision_Currently;Over the years', 
                   'vision_Over the years', 'anxiety_Currently', 'anxiety_Over the years', 'Headaches_Currently',
                   'Headaches_Over the years', 'dizziness_Over the years']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later

data['E']=1   #all pedestrians cross, so no right censored data

data=data.apply(pd.to_numeric, errors='coerce')

In [ ]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);

In [134]:
def calculate_vif_(X, thresh=10.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [136]:
data=calculate_vif_(data, thresh=5.0)

/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


dropping 'Full Braking Before Impact_-1.0' at index: 6
dropping 'Clear' at index: 9
dropping 'One way' at index: 10
dropping 'Day' at index: 12
dropping 'Age_9-12' at index: 14
dropping 'Age_18 - 24' at index: 16
dropping 'Occupation_Employed' at index: 22
dropping 'Occupation_kid' at index: 24
dropping 'Education_Bachelors degree' at index: 24
dropping 'Education_kid' at index: 29
dropping 'mode_Bike' at index: 30
dropping 'workwalk_No' at index: 33
dropping 'shopwalk_No' at index: 35
dropping 'E' at index: 49
dropping 'AV' at index: 5
dropping 'Lane Width' at index: 2
dropping 'Speed Limit' at index: 1
dropping 'driving license_Yes' at index: 26
dropping 'Minimum Gap' at index: 1
dropping 'Mean Arrival Rate' at index: 1
dropping 'mode_Car' at index: 24
dropping 'numcars' at index: 8
dropping 'Heart_Over the years' at index: 32
Remaining variables:
Index(['Wait Time (s)', 'Full Braking Before Impact_1',
       'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
  

In [137]:
data['E']=1
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


<lifelines.CoxPHFitter: fitted with 2463 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2463
  number of events = 2463
    log-likelihood = -16635.83
  time fit was run = 2019-03-11 23:54:38 UTC

---
                                      coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Full Braking Before Impact_1          0.07       1.08      0.09  0.82   0.41      1.29       -0.10        0.25
Full Braking Before Impact_2          0.01       1.01      0.09  0.12   0.91      0.14       -0.16        0.18
Full Braking Before Impact_3         -0.01       0.99      0.09 -0.15   0.88      0.18       -0.19        0.16
Snowy                                -0.06       0.94      0.04 -1.45   0.15      2.76       -0.15        0.02
two way                               0.07       1.08      0.05  1.46   0.14      2.80       -0.02        0.17
Two way with median                   0.23       1.26      0.05  4.48 <0

In [ ]:
data=data.drop(columns=['Full Braking Before Impact_1','Full Braking Before Impact_3','Full Braking Before Impact_2','Night',
                        'Age_12-15','Age_25 - 29','Education_Professional degree','Education_College/University student',
                        'Headaches_Currently','Occupation_Unemployed',
                        'workwalk_Sometimes','shopwalk_kid','Heart_Currently','vision_Currently;Over the years','anxiety_Currently'],axis=1)

In [142]:
cph = CoxPHFitter()
cph.fit(data, 'Wait Time (s)', event_col='E')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 2463 observations, 0 censored>
      duration col = 'Wait Time (s)'
         event col = 'E'
number of subjects = 2463
  number of events = 2463
    log-likelihood = -16639.21
  time fit was run = 2019-03-11 23:59:40 UTC

---
                               coef  exp(coef)  se(coef)     z      p  -log2(p)  lower 0.95  upper 0.95
Snowy                         -0.07       0.93      0.04 -1.54   0.12      3.02       -0.15        0.02
two way                        0.07       1.08      0.05  1.50   0.13      2.90       -0.02        0.17
Two way with median            0.23       1.25      0.05  4.48 <0.005     17.06        0.13        0.32
Age_15-18                     -0.29       0.75      0.18 -1.67   0.09      3.41       -0.64        0.05
Age_30 - 39                    0.33       1.40      0.06  5.51 <0.005     24.76        0.21        0.45
Age_40 - 49                    0.38       1.46      0.10  3.85 <0.005     13.03        0.19        0.58
Age_50 - 5

In [66]:
data.shap

(2463, 63)